# KFold를 이용한 교차검정

###### 총 데이터 수가 적은 경우 사용한다.
###### 데이터를 k개로 나누고 `k-1을 train` `하나를 test`로 둔다.
###### 첫 번째 덩어리를 test로->두 번째 덩어리를 test로->...->k 번째 덩어리를 test로 두며 지도학습을 한다.
###### KFold를 적용하지 않을 때는 한 번인 과정이 KFold를 적용하면 k번 반복되는 과정으로 바뀐다.
###### 이 과정을 통해 적은 데이터를 많이 부풀려 사용할 수 있다.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

iris=load_iris()
features=iris.data
label=iris.target

features.shape

(150, 4)

In [2]:
#DecisionTreeClassifier 객체 생성
dt_clf=DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 KFold 객체 생성
kfold=KFold(n_splits=5)

#폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy=[]

In [3]:
#폴드 별 학습용, 검증용 데이터 세트의 행 인덱스 확인
for train_index,test_index in kfold.split(features) : 
    print(train_index,test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [4]:
import numpy as np
for train_index,test_index in kfold.split(features):
    X_train,X_test=features[train_index],features[test_index]
    y_train,y_test=label[train_index],label[test_index]
    
    dt_clf.fit(X_train,y_train)
    pred=dt_clf.predict(X_test)
    
    acc= np.round(accuracy_score(y_test,pred),3)
    train_size=X_train.shape[0]
    test_size=X_test.shape[0]
    
    print("정확도: %f, 학습데이터 크기: %d, 검증데이터 크기: %d" %(acc,train_size,test_size))
    cv_accuracy.append(acc)

print("평균 검증 정확도: ",np.mean(cv_accuracy))

정확도: 1.000000, 학습데이터 크기: 120, 검증데이터 크기: 30
정확도: 0.967000, 학습데이터 크기: 120, 검증데이터 크기: 30
정확도: 0.867000, 학습데이터 크기: 120, 검증데이터 크기: 30
정확도: 0.933000, 학습데이터 크기: 120, 검증데이터 크기: 30
정확도: 0.733000, 학습데이터 크기: 120, 검증데이터 크기: 30
평균 검증 정확도:  0.9


# StratifiedKFold를 이용한 교차검정

###### 불균형한 데이터셋을 위한 방법이다.
###### 예를 들면 True_label=10000 False_label=10 인 경우이다.
###### 이 경우, KFold로 나누면 모든 값이 True_label이 존재하는 경우가 생길 것이다.
###### 그렇다면 모든 것이 True_label가 되므로 정확도가 떨어진다.
###### 따라서 이 때 StratifiedKFold를 사용한다.
#### 간단히 말하자면 StratifiedKFold는 target의 속성의 개수를 같게 해주는 기법이다.

In [5]:
import pandas as pd
iris_data=load_iris()
iris_df=pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
iris_df['label']=iris_data.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [6]:
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [7]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits =3)
n = 0

for train_index, text_index in skf.split(iris_df, iris_df['label']):
    n = n+1
    
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print("[교차검증 %d]" % n)
    print('학습용 레이블 분포: \n', label_train.value_counts())
    print('검증용 레이블 분포: \n', label_test.value_counts())

[교차검증 1]
학습용 레이블 분포: 
 2    34
0    33
1    33
Name: label, dtype: int64
검증용 레이블 분포: 
 2    30
Name: label, dtype: int64
[교차검증 2]
학습용 레이블 분포: 
 1    34
0    33
2    33
Name: label, dtype: int64
검증용 레이블 분포: 
 2    30
Name: label, dtype: int64
[교차검증 3]
학습용 레이블 분포: 
 0    34
1    33
2    33
Name: label, dtype: int64
검증용 레이블 분포: 
 2    30
Name: label, dtype: int64


In [8]:
# DTC 객체 생성
dt_clf = DecisionTreeClassifier(random_state=156)

# 3개의 폴드 세트로 분리하는 StratifiedKFold 객체 생성
skfold = StratifiedKFold(n_splits=3)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []
n = 0

In [9]:
n=0

iris=load_n = 0

# 폴드별 학습용, 검증용 데이터 세트의 행 인덱스 확인
for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train = label[train_index]
    y_test = label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복시마다 정확도 측정
    n +=1
    acc = np.round(accuracy_score(y_test, pred), 3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('%d \n 정확도 : %f, 학습데이터 크기 : %d, 검증데이터 크기 : %d' %(n, acc, train_size, test_size))
    cv_accuracy.append(acc)

print('평균 검증 정확도 : ', np.mean(cv_accuracy).round(1))

1 
 정확도 : 0.980000, 학습데이터 크기 : 100, 검증데이터 크기 : 50
2 
 정확도 : 0.940000, 학습데이터 크기 : 100, 검증데이터 크기 : 50
3 
 정확도 : 0.980000, 학습데이터 크기 : 100, 검증데이터 크기 : 50
평균 검증 정확도 :  1.0


---------------------------------------------------------------------

###### StratifiedKFold는 target의 속성의 개수를 같게 해주는 기법이므로 정확도가 높다.
###### 실제로 평균 검증 정확도를 KFold와 비교해보면 KFold보다 정확도가 높은 것을 확인할 수 있다